In [46]:
import numpy as np

# Zadatak

In [160]:
Cij = np.array(
    [
     [1,4,6],
     [8,2,5],
     [3,7,9],
    ]
)

kapaciteti_ishodista = np.array([15,45,30])
potrebe_odredista = np.array([20,20,50])

# Originalni oblik transportnog problema

In [ ]:
# Nekakav stackoverflow hack za subscript https://stackoverflow.com/a/24392215
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
Z = "Funkcija cilja:\nZ = \n"
ogranicenja_redova = "Ograničenja redova:\n"
ogranicenja_stupaca = "Ograničenja stupaca:\n"

for i in range(0, len(Cij)):
  for j in range(0, len(Cij[i])):
    trosak = Cij[i][j]
    teret = f"x{i+1}{j+1}".translate(SUB)
    ogranicenja_redova += f"{teret}+"
    Z += f"{trosak}{teret} + "
  ogranicenja_redova = ogranicenja_redova[:-1]
  ogranicenja_redova += f" = {kapaciteti_ishodista[i]}\n"
  Z += "\n"

for j in range(len(Cij[0])):
  for i in range(len(Cij[1])):
    teret = f"x{i+1}{j+1}".translate(SUB)
    ogranicenja_stupaca += f"{teret}+"
  ogranicenja_stupaca = ogranicenja_stupaca[:-1]
  ogranicenja_stupaca += f" = {potrebe_odredista[j]}\n"

# Makni zadnji plus i \n
Z = Z[:-3]
Z += "\n->min"

print(Z)
print()
print(ogranicenja_redova)
print()
print(ogranicenja_stupaca)
print("Xij>=0")




Funkcija cilja:
Z = 
5x₁₁ + 10x₁₂ + 15x₁₃ + 
12x₂₁ + 4x₂₂ + 8x₂₃ + 
7x₃₁ + 3x₃₂ + 9x₃₃ + 
14x₄₁ + 16x₄₂ + 1x₄₃ 
->min

Ograničenja redova:
x₁₁+x₁₂+x₁₃ = 45
x₂₁+x₂₂+x₂₃ = 50
x₃₁+x₃₂+x₃₃ = 90
x₄₁+x₄₂+x₄₃ = 45


Ograničenja stupaca:
x₁₁+x₂₁+x₃₁ = 120
x₁₂+x₂₂+x₃₂ = 40
x₁₃+x₂₃+x₃₃ = 70

Xij>=0


# Dualni oblik transportnog problema

In [ ]:
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
Z = "Funkcija cilja:\nZd = \n"
ogranicenja = ""

for i, kapacitet in enumerate(kapaciteti_ishodista):
  index = str(i+1).translate(SUB)
  Z += f"{kapacitet}u{index} + " 
Z += "\n"
for i, kapacitet in enumerate(potrebe_odredista):
  index = str(i+1).translate(SUB)
  Z += f"{kapacitet}v{index} + " 

for i in range(len(Cij)):
  for j in range(len(Cij[i])):
    ogranicenja += f"u{i+1}+v{j+1}".translate(SUB) + f" <= {Cij[i][j]}\n"
  ogranicenja += "\n"

Z = Z[:-1]
Z += "\n->max"
print(Z)
print()
print(ogranicenja)

Funkcija cilja:
Zd = 
45u₁ + 50u₂ + 90u₃ + 45u₄ + 
120v₁ + 40v₂ + 70v₃ +
->max

u₁+v₁ <= 5
u₁+v₂ <= 10
u₁+v₃ <= 15

u₂+v₁ <= 12
u₂+v₂ <= 4
u₂+v₃ <= 8

u₃+v₁ <= 7
u₃+v₂ <= 3
u₃+v₃ <= 9

u₄+v₁ <= 14
u₄+v₂ <= 16
u₄+v₃ <= 1




# SZ metoda


In [48]:
# Kreiraj matricu tereta koja ima iste dimenzije kao i Cij i popuni ju s nulama
tereti = np.zeros(Cij.shape)
# Ovdje ce se biljeziti index stupca koji je zadnji popunjen
stupac_preskakanja = -1

for i in range(len(Cij)):
  for j in range(len(Cij[i])):
    # Preskoci ovaj stupac ako mu je kapacitet vec popunjen
    if stupac_preskakanja != -1 and stupac_preskakanja >= j:
      continue

    # Izracunaj popunjenosti trenutnog reda i stupca
    popunjenost_reda = np.sum(tereti[i])
    popunjenost_stupca = np.sum(tereti[:,j])

    # Izracunaj koliko je dozvoljeno popuniti red a koliko stupac
    dozvoljeno_popunjavanje_red = kapaciteti_ishodista[i] - popunjenost_reda
    dozvoljeno_popunjavanje_stupac = potrebe_odredista[j] - popunjenost_stupca

    # Uzmi manju dopunu kao dozvoljenu
    popunjivo = np.min((dozvoljeno_popunjavanje_red, dozvoljeno_popunjavanje_stupac))

    # Dodaj u teret u matricu tereta
    tereti[i][j] = popunjivo

    # Dodaj novi teret u sume popunjenosti reda i stupca
    popunjenost_reda += popunjivo
    popunjenost_stupca += popunjivo

    # Oznaci index stupca koji je popunjen da bi se preskocio
    # svaki put kad iteracija dodje u novi red
    if popunjenost_stupca == potrebe_odredista[j]:
      stupac_preskakanja = j

    # Ako je red popunjen idi u sljedeci
    if popunjenost_reda == kapaciteti_ishodista[i]:
      break

print("Matrica tereta:")
print(tereti)

Matrica tereta:
[[28.  2.  0.]
 [ 0. 20.  0.]
 [ 0. 15. 25.]]


# Metoda minimalnih troškova

In [83]:
tereti = np.zeros(Cij.shape)
popunjeni_stupci = []
popunjeni_redovi = []

while True:
  # Ovdje ce se racunati popunjivost za sva ona polja koja su jos u igri
  # tj. nisu u popunjenim stupcima ili redovima
  # key = stvarni trosak (int)
  # value = {"i": int, "j": int, "teret": int}
  izracuni = {}

  for i in range(len(Cij)):
    if not i in popunjeni_redovi:
      for j in range(len(Cij[i])):
        if not j in popunjeni_stupci:
          # Izracunaj popunjenosti trenutnog reda i stupca
          popunjenost_reda = np.sum(tereti[i])
          popunjenost_stupca = np.sum(tereti[:,j])

          # Izracunaj koliko je dozvoljeno popuniti red a koliko stupac
          dozvoljeno_popunjavanje_red = kapaciteti_ishodista[i] - popunjenost_reda
          dozvoljeno_popunjavanje_stupac = potrebe_odredista[j] - popunjenost_stupca

          # Uzmi manju dopunu kao dozvoljenu
          popunjivo = np.min((dozvoljeno_popunjavanje_red, dozvoljeno_popunjavanje_stupac))

          trosak = Cij[i][j]
          # Dodaj trosak i zapamti njegove indexe i popunjivost
          if not trosak in izracuni:
            izracuni[trosak] = {"i": i, "j": j, "teret": popunjivo}
          # Ako je polje s ovim stvarnim troskom vec bilo,
          # stavi ga samo ako je njegov teret manji
          elif izracuni[trosak]["teret"] < popunjivo:
            izracuni[trosak] = {"i": i, "j": j, "teret": popunjivo}

  # Uzmi polje s najmanjim stvarnim troskom
  najmanji = np.array(list(izracuni.keys())).min()

  red = izracuni[najmanji]["i"]
  stupac = izracuni[najmanji]["j"]
  teret = izracuni[najmanji]["teret"]

  # Dodaj teret koji se moze na to polje staviti u matricu tereta
  tereti[red][stupac] = teret

  # Ako je trenutni red pun, oznaci ga kao pun
  if np.sum(tereti[red]) == kapaciteti_ishodista[red]:
    popunjeni_redovi.append(red)
  # Ako je trenutni stupac pun, oznaci ga kao pun
  if np.sum(tereti[:,stupac]) == potrebe_odredista[stupac]:
    popunjeni_stupci.append(stupac)

  # Ako su svi redovi i stupci popunjeni gotovo je
  if len(popunjeni_redovi) == len(kapaciteti_ishodista) or len(popunjeni_stupci) == len(potrebe_odredista):
    break

print("Matrica tereta:")
print(tereti)

Matrica tereta:
[[10.  0. 15.]
 [20.  0.  0.]
 [ 0. 25.  5.]]


# Vogelova metoda

In [167]:
tereti = np.zeros(Cij.shape)
_Cij = np.copy(Cij)
it = 0

while True:
  # Postavi sve na -1
  ri = np.full(kapaciteti_ishodista.shape, -1)
  rj = np.full(potrebe_odredista.shape, -1)

  # Izracunaj ri za redove
  for i in range(len(_Cij)):
    if not np.all(_Cij[i] == -1):
      sortirano = np.sort(_Cij[i])
      if np.count_nonzero(sortirano > -1) >= 2:
        min, min2, *_ = [x for x in sortirano if x>-1]
        ri[i] = abs(min-min2)
      else:
        ri[i] = 1

  # Izracunaj rj za stupce
  for j in range(len(_Cij[0])):
    if not np.all(_Cij[:,j] == -1):
      sortirano = np.sort(_Cij[:,j])
      if np.count_nonzero(sortirano > -1) >= 2:
        min, min2, *_ = [x for x in sortirano if x>-1]
        rj[j] = abs(min-min2)
      else:
        rj[j] = 1

  # Pronadji max ri i max rj
  #max_i = np.argwhere(ri == np.amax(ri))
  i = np.argmax(ri)
  j = np.argmax(rj)
  # Izracunaj drugi index da se zna na koje polje se stavlja teret
  if ri[i] > rj[j]:
    j = np.argmin(np.ma.masked_less(_Cij[i], 0))
  else:
    i = np.argmin(np.ma.masked_less(_Cij[:,j], 0))

  # Izracunaj koliko je dozvoljeno popuniti red a koliko stupac
  dozvoljeno_popunjavanje_red = kapaciteti_ishodista[i] - np.sum(tereti[i])
  dozvoljeno_popunjavanje_stupac = potrebe_odredista[j] - np.sum(tereti[:,j])

  # Uzmi manju dopunu kao dozvoljenu
  popunjivo = np.min((dozvoljeno_popunjavanje_red, dozvoljeno_popunjavanje_stupac))
  tereti[i][j] = popunjivo

  # Ako je trenutni red pun, oznaci ga kao pun
  if np.sum(tereti[i]) == kapaciteti_ishodista[i]:
    _Cij[i] = -1
  # Ako je trenutni stupac pun, oznaci ga kao pun
  if np.sum(tereti[:,j]) == potrebe_odredista[j]:
    _Cij[:,j] = -1
  # Ako su svi redovi i stupci popunjeni gotovo je
  if np.all(_Cij == -1):
    break

print("Matrica tereta:")
print(tereti)

Matrica tereta:
[[ 0.  0. 15.]
 [ 0. 20. 25.]
 [20.  0. 10.]]
